# Threshold Spreading

In [189]:
# Imports

import numpy as np
import networkx as nx

In [190]:
G = nx.read_edgelist('M1/edges.csv', delimiter = ',')
print(G)

Graph with 1429 nodes and 19357 edges


We first create a function which computes the new infected nodes using threshold reinforcement:

In [191]:
# Modified SI function using threshold
def new_infected_threshold(G, I, kappa, t):
    '''SI function which uses thresholds to determine which new 
    nodes are infected by their neighbours in one step of the infection process.
    
    Args:
        G (nx.Graph): Graph to perform algorithm on
        I (set): Set of already infected nodes
        kappa (int): Threshold
        t (int): Time step of current infection step

    Return:
        new_infected (set): Set of newly infected nodes.
    '''

    # Create a new set of the newly infected nodes
    new_infected = set()

    # Check if time step t is 1 and set threshold to 1 if true
    if t == 1:
        kappa = 1

    # Iterate over all nodes in the network which are not already infected
    for u in set(G.nodes) - I:

        # Check if the amount of infected neighbors passes the threshold
        if len(set(G.neighbors(u)) & I) >= kappa:
            new_infected.add(u)

    return new_infected

We now create a function to run a complete infection process on the network:

In [192]:
def threshold_infection_process(G, s, kappa):
    '''Perform SI infection process on a complete network using threshold reinforcement.
    
    Args:
        G (nx.Graph): Graph to perform algorithm on
        s (int): Starting node for the infection process
        kappa (int): Threshold

    Return:
        I (set): Set of infected nodes
        t (int): Time step reached when spreading stopped
    '''

    I = {s} # Set of infected nodes, starts with only staring node infected
    t = 0 # Initialize time step

    # Time step loop, continues until all nodes are infected
    while len(I) < len(G.nodes):

        # Increment t
        t += 1

        # Find new infected nodes
        new_infected = new_infected_threshold(G, I, kappa, t)

        # # Print for debugging
        # print(f"Time step {t}: Newly infected nodes: {new_infected}")

        # Break if no new nodes are infected
        if len(new_infected) == 0:
            break

        # Add new infected nodes to I
        I = I.union(new_infected)

        # # Print for debugging
        # print(f"Time step {t}: Infected nodes: {I}")

    return I, t

We now check the infection results for different values of kappa:

In [195]:
# Keep list of kappa threshold values
thresholds = list(range(1, 11))

# Iterate over each threshold value
for kappa in thresholds:

    node_results = {} # Keep dictionary of results for each node, will be dict of dicts
    success_rate = .0 # Ratio of successfull infections
    V = len(G.nodes) # Amount of nodes

    # Keep track of best and worst starting nodes along with their amounts of steps taken, lower step count = better
    best_starting_nodes = [] # Initialize as list to keep option of multiple best starting nodes
    best_steps_taken = float('inf') # Initialize with positive infinity
    worst_starting_nodes = [] # Initialize as list to keep option of multiple worst starting nodes
    worst_steps_taken = 0 # Initialize with 0

    # Iterate over all possible starting nodes
    for s in G.nodes:
        
        I, steps_taken = threshold_infection_process(G, s, kappa) # Run infection process
        success = len(I) == V # Keep bool for whether infection was successfull
        node_results[s] = {'successfull': success, 'steps_taken': steps_taken, 'amount_infected': len(I)} # Add dictionary of results for node s to node_results
        success_rate += success / V # Increment success_rate

        # Update best and worst starting nodes
        if steps_taken == best_steps_taken: # Same as current best, add to list
            best_starting_nodes.append(s)
        elif steps_taken < best_steps_taken: # Better than current best, remove all others from list
            best_starting_nodes = [s]
            best_steps_taken = steps_taken

        if steps_taken == worst_steps_taken: # Same as current worst, add to list
            worst_starting_nodes.append(s)
        elif steps_taken > worst_steps_taken: # Worse than current worst, remove all others from list
            worst_starting_nodes = [s]
            worst_steps_taken = steps_taken

    # Compute statistics for the amount of steps taken

    # All infection passes
    average_steps_total = sum(node_results[u]['steps_taken'] for u in node_results) / V

    # Successfull infection passes
    steps_taken_successfull = [node_results[u]['steps_taken'] for u in node_results if node_results[u]['successfull']]
    if len(steps_taken_successfull) == 0: # If no successfull infections, set average steps taken to 0 to avoid division by 0
        average_steps_successfull = 0
    else:
        average_steps_successfull = sum(steps_taken_successfull) / len(steps_taken_successfull)

    # Unsuccessfull infection passes
    steps_taken_failure = [node_results[u]['steps_taken'] for u in node_results if not node_results[u]['successfull']]
    if len(steps_taken_failure) == 0: # If no failed infections, set average steps taken to 0 to avoid division by 0
        average_steps_failure = 0
    else:
        average_steps_failure = sum(steps_taken_failure) / len(steps_taken_failure)
    
    # Print statistics

    print(f'Kappa = {kappa}:')
    print(f'Success rate: {success_rate:.4f}')

    if success_rate < 1:
        average_nodes_infected = sum(node_results[u]['amount_infected'] for u in node_results) / V
        print(f'Average amount of nodes infected: {average_nodes_infected:.4f} / {V}')

    print('Average steps taken:')
    print(f'Total: {average_steps_total:.4f}, Successfull: {average_steps_successfull:.4f}, Failure: {average_steps_failure:.4f}')

    print(f'Best Starting Node(s): {best_starting_nodes} with {best_steps_taken} steps taken')
    print(f'Worst Starting Node(s): {worst_starting_nodes} with {worst_steps_taken} steps taken \n')

Kappa = 1:
Success rate: 1.0000
Average steps taken:
Total: 5.1896, Successfull: 5.1896, Failure: 0.0000
Best Starting Node(s): ['1', '5', '17', '22', '28', '29', '33', '34', '38', '42', '52', '66', '172', '537', '120', '80', '560', '330', '573', '75', '590', '74', '117', '81', '200', '556', '571', '371', '366', '167', '570', '346', '848', '73', '613', '475', '67', '68', '725', '83', '679', '760', '440', '634', '145', '78', '263', '157', '1310'] with 4 steps taken
Worst Starting Node(s): ['906', '1250'] with 7 steps taken 

Kappa = 2:
Success rate: 0.9993
Average amount of nodes infected: 1428.0028 / 1429
Average steps taken:
Total: 6.9405, Successfull: 6.9440, Failure: 2.0000
Best Starting Node(s): ['486'] with 2 steps taken
Worst Starting Node(s): ['1177', '809'] with 11 steps taken 

Kappa = 3:
Success rate: 0.0000
Average amount of nodes infected: 1362.4556 / 1429
Average steps taken:
Total: 10.5745, Successfull: 0.0000, Failure: 10.5745
Best Starting Node(s): ['48', '1296', '1398'

KeyboardInterrupt: 